In [1]:
import pandas as pd
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

## Analizamos cómo agrupar interacciones por CIF_ID

In [2]:
df = pd.read_feather("interacciones.feather")

In [3]:
df.head(5)

,CIF_ID,IN_OUT,FECHA
0,19385529.0,O,1/11/2018
1,13890033.0,O,1/11/2018
2,13890033.0,O,1/11/2018
3,2156057.0,O,1/11/2018
4,18472299.0,O,1/11/2018


In [4]:
df["IN_OUT"].value_counts(dropna=False)

O    24374873
I    11751905
A     1199592
Name: IN_OUT, dtype: int64

In [5]:
df = df[df["IN_OUT"].isin(['O','I','A'])]

In [6]:
df["FECHA"] = df["FECHA"].str.slice(stop=10) 

In [7]:
df.loc[df["FECHA"].str.contains(" [0-9]", na=False), "FECHA"] = df.loc[df["FECHA"].str.contains(" [0-9]", na=False), "FECHA"].str.slice(stop=8) 

In [8]:
df["FECHA"] = df["FECHA"].str.replace(" ","")

In [9]:
def to_date(s):
    dates = {date:pd.to_datetime(date, format="%d/%m/%Y") for date in s.unique()}
    return s.map(dates)

df["periodo"] = to_date(df["FECHA"])

Transformamos la fecha en anio y mes

In [10]:
def to_yearmonth(s):
    dates = {date:pd.Timestamp(date).strftime('%Y-%m') for date in s.unique()}
    return s.map(dates)

df["periodo"] = to_yearmonth(df["periodo"].dropna())

In [11]:
df["periodo"].value_counts().sort_index().plot.bar()

In [12]:
data = {fecha:i + 1 for i,fecha in enumerate(df["periodo"].unique())}
data

{'2018-11': 1,
 '2018-12': 2,
 '2019-01': 3,
 '2019-02': 4,
 '2019-03': 5,
 '2019-04': 6,
 '2019-05': 7,
 '2019-06': 8,
 '2019-07': 9,
 '2019-08': 10,
 '2019-09': 11,
 '2019-10': 12,
 '2019-11': 13}

In [13]:
df["periodo_int"] = df["periodo"].map(data)

In [14]:
df = df[df["periodo_int"] < 19]

In [15]:
df.head()

,CIF_ID,IN_OUT,FECHA,periodo,periodo_int
0,19385529.0,O,1/11/2018,2018-11,1
1,13890033.0,O,1/11/2018,2018-11,1
2,13890033.0,O,1/11/2018,2018-11,1
3,2156057.0,O,1/11/2018,2018-11,1
4,18472299.0,O,1/11/2018,2018-11,1


In [16]:
to_pivot = df[["CIF_ID","IN_OUT","periodo_int"]]

In [17]:
pivoted = to_pivot.pivot_table(index=["CIF_ID"], columns=["periodo_int","IN_OUT"], aggfunc="size")
pivoted

periodo_int   1                 2                  3                  4        \
IN_OUT         A    I       O    A     I       O    A     I       O    A    I   
CIF_ID                                                                          
0.0          NaN  NaN  2029.0  NaN   NaN  1206.0  NaN   NaN  1394.0  NaN  NaN   
137.0        NaN  NaN     NaN  NaN   NaN     NaN  NaN   NaN     NaN  NaN  NaN   
162.0        NaN  NaN     1.0  NaN   NaN     1.0  NaN   1.0     1.0  NaN  NaN   
240.0        NaN  NaN     NaN  NaN   NaN     NaN  NaN   NaN     NaN  NaN  NaN   
269.0        NaN  NaN     NaN  NaN   NaN     NaN  NaN   NaN     NaN  NaN  NaN   
571.0        NaN  2.0     2.0  NaN   NaN     2.0  NaN   NaN     1.0  NaN  2.0   
573.0        NaN  NaN     1.0  NaN   NaN     NaN  NaN   NaN     1.0  NaN  NaN   
1009.0       NaN  NaN     NaN  NaN   NaN     1.0  NaN   NaN     1.0  NaN  NaN   
1075.0       NaN  NaN     NaN  NaN   NaN     NaN  NaN   NaN     NaN  NaN  NaN   
1103.0       NaN  NaN     1.0  NaN   NaN     1.0  NaN   NaN     NaN  NaN  NaN   
1148.0       NaN  NaN     NaN  NaN   NaN     NaN  NaN   NaN     NaN  NaN  NaN   
1313.0       NaN  NaN     4.0  NaN   NaN     3.0  NaN   NaN     2.0  NaN  NaN   
1429.0       NaN  1.0     2.0  NaN   1.0     4.0  NaN   2.0     4.0  2.0  4.0   
1482.0       NaN  NaN     4.0  NaN   NaN     4.0  NaN   NaN     3.0  NaN  NaN   
1559.0       NaN  NaN     4.0  NaN   NaN     3.0  NaN   NaN     2.0  NaN  NaN   
1774.0       NaN  NaN     3.0  NaN   NaN     1.0  NaN   1.0     2.0  NaN  NaN   
1779.0       NaN  NaN     NaN  NaN   NaN     NaN  1.0  13.0    10.0  NaN  NaN   
1923.0       NaN  NaN     1.0  NaN   NaN     1.0  NaN   NaN     1.0  NaN  NaN   
2010.0       NaN  NaN     NaN  NaN   3.0     NaN  NaN   NaN     NaN  NaN  NaN   
2390.0       NaN  NaN     NaN  NaN   NaN     NaN  NaN   NaN     NaN  NaN  NaN   
2537.0       NaN  NaN     1.0  NaN   NaN     1.0  NaN   NaN     1.0  NaN  NaN   
2643.0       NaN  NaN     2.0  NaN   NaN     NaN  NaN   NaN     1.0  NaN  NaN   
2994.0       NaN  2.0     2.0  NaN   1.0     1.0  NaN   NaN     2.0  NaN  1.0   
3309.0       NaN  NaN     NaN  NaN   NaN     NaN  NaN   NaN     NaN  NaN  3.0   
3426.0       NaN  NaN     NaN  1.0  28.0     4.0  4.0   4.0     1.0  NaN  1.0   
3444.0       NaN  NaN     NaN  NaN   NaN     NaN  NaN   NaN     NaN  NaN  NaN   
3454.0       NaN  NaN     1.0  NaN   NaN     1.0  NaN   NaN     NaN  NaN  NaN   
3867.0       NaN  NaN     2.0  NaN   NaN     2.0  NaN   NaN     2.0  NaN  NaN   
4107.0       NaN  NaN     NaN  NaN   NaN     NaN  NaN   NaN     NaN  NaN  NaN   
4408.0       NaN  NaN     2.0  NaN   NaN     1.0  NaN   NaN     1.0  NaN  NaN   
4431.0       NaN  NaN     2.0  NaN   NaN     1.0  NaN   2.0     1.0  NaN  NaN   
4445.0       NaN  NaN     1.0  NaN   NaN     1.0  NaN   NaN     1.0  NaN  NaN   
4598.0       NaN  NaN     NaN  NaN   NaN     1.0  NaN   NaN     1.0  NaN  NaN   
4655.0       NaN  NaN     4.0  NaN   NaN     NaN  NaN   NaN     3.0  NaN  NaN   
5049.0       NaN  3.0     3.0  NaN   NaN     1.0  NaN   NaN     1.0  NaN  NaN   
5067.0       NaN  NaN     NaN  1.0   7.0     1.0  NaN   5.0     5.0  NaN  NaN   
5096.0       NaN  1.0     7.0  NaN   NaN     2.0  NaN   2.0     1.0  NaN  1.0   
5142.0       NaN  NaN     NaN  NaN   NaN     NaN  NaN   NaN     NaN  NaN  1.0   
5202.0       NaN  NaN     NaN  NaN   NaN     NaN  NaN   NaN     NaN  NaN  NaN   
5231.0       NaN  NaN     3.0  NaN   1.0     1.0  NaN   NaN     2.0  NaN  NaN   
5332.0       NaN  NaN     4.0  NaN   NaN     2.0  NaN   NaN     4.0  NaN  1.0   
5547.0       NaN  NaN     NaN  NaN   NaN     NaN  NaN   NaN     NaN  NaN  NaN   
5575.0       NaN  NaN     NaN  NaN   NaN     NaN  NaN   NaN     NaN  NaN  NaN   
5899.0       NaN  NaN     NaN  NaN   NaN     NaN  NaN   NaN     NaN  NaN  NaN   
5915.0       NaN  NaN     1.0  NaN  11.0     4.0  NaN   NaN     2.0  NaN  2.0   
5937.0       NaN  NaN     1.0  NaN   NaN     1.0  NaN   NaN     NaN  NaN  NaN   
5976.0       NaN  NaN     NaN  

In [18]:
df_interacciones = pd.DataFrame(pivoted.to_records())

In [19]:
df_interacciones = df_interacciones.rename(columns=lambda x: x.replace("(","").replace(")","").replace(", ","_TIPOINT_").replace("'","")) 

In [20]:
df_interacciones.reset_index(drop=True).to_feather("interacciones_tipo_periodo_x_cif_id.feather")

In [21]:
df_interacciones.shape

(1636467, 40)